<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/notebook-testing-ivy/blob/main/Testing_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [315]:
import torch
import tensorflow as tf

In [2]:
a = torch.tensor([1, 2, 3, 4])
b = torch.tensor([[5, 6], [7, 8]])
print(a.reshape(b.shape))
print(torch.reshape(a, b.shape))

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])


In [332]:
def gather(
    params: torch.Tensor,
    indices: torch.Tensor,
    /,
    *,
    axis: int = -1,
    batch_dims: int = 0,
    out: torch.Tensor = None,
) -> torch.Tensor:
    axis %= len(params.shape)
    axis = abs(len(params.shape) + axis) if axis < 0 else axis
    print(f"AXIS: {axis}")
    batch_dims %= len(params.shape)
    result = []

    def expand_p_i(params, indices, axis = axis):
        ind_dim_helper_table = [1 for dim in list(indices.shape)]
        ind_singleton_dims = torch.Size(ind_dim_helper_table)
        param_singleton_dims_table = [1 for dim in list(params.shape)]
        param_singleton_dims = torch.Size(param_singleton_dims_table)
        print("INDICES SHAPE: ", indices.shape)
        print("PARAMS SHAPE: ", params.shape)
        print(f"SINGLETON_DIMS: {param_singleton_dims}, {ind_singleton_dims}")
        print("RESHAPE SHAPE: ", param_singleton_dims[:axis]
                                     + indices.shape[batch_dims:]
                                     + param_singleton_dims[axis + 1:], "\n",(params.shape[:axis]
                                   + ind_singleton_dims[batch_dims: - 1] + max([ind_singleton_dims[-1:], params.shape[axis:axis + 1]])
                                   + params.shape[axis + 1:]))
        print(param_singleton_dims[:axis],
                                      indices.shape[batch_dims:],
                                      param_singleton_dims[axis + 1:], batch_dims)
        print("SIZE TUPLES: ", indices.shape[batch_dims:], params.shape[axis:axis + 1], params.shape, params)
        params_insert_shape = ((torch.tensor(indices.shape[batch_dims:]) == params.shape[axis % len(params)]) * params.shape[axis % len(params)]).long() + (torch.tensor(indices.shape[batch_dims:]) != params.shape[axis % len(params)]).long()
        params_insert_shape = [dim for dim in params_insert_shape]
        print(torch.Size(params_insert_shape) != torch.Size([0]))
        params_insert_shape = torch.Size(params_insert_shape) if torch.Size(params_insert_shape) != torch.Size([0]) else indices.shape[batch_dims:]
        print(f"\nParams insert shape: {params_insert_shape}", " params reshape shape: ", (params.shape[:axis]
                                   + params_insert_shape
                                   + params.shape[axis + 1:]), "Params shape: ", params.shape)
        indices_ex = indices if (params.dim() <= 1) else \
                    indices.reshape(param_singleton_dims[:axis]
                                     + indices.shape[batch_dims:]
                                     + param_singleton_dims[axis + 1:])
        params_shape = list(params.shape[:axis]
                                   + params_insert_shape
                                   + params.shape[axis + 1:])
        if params.shape[axis % len(params)] not in params_shape:
          params_shape[-1] = params.shape[axis]
        params_shape = torch.Size(params_shape)


        params_ex = params if indices_ex.dim() == params.dim() else params.reshape(params_shape)

        print("FINAL SHAPE: ", params.shape[:axis]
                                         + indices.shape[batch_dims:]
                                         + params.shape[axis + 1:], params_ex.shape, indices_ex.shape)

        params_ex_mask = torch.tensor(params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:]) * (torch.tensor(params_ex.shape) == 1)
        print(params_ex_mask, (torch.tensor(params_ex.shape) == 1))
        print("CONDITIONS FOR MASKS- PARAMS: ", (params_ex_mask != (torch.tensor(params_ex.shape) == 1).flatten()))
        if any((params_ex_mask != (torch.tensor(params_ex.shape) == 1).flatten())):
            params_ex = params_ex.expand(params.shape[:axis]
                                         + indices.shape[batch_dims:]
                                         + params.shape[axis + 1:])
        indices_ex_mask = torch.tensor(params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:]) * (torch.tensor(indices_ex.shape) == 1)
        print("CONDITIONS FOR MASKS- INDICES: ", (indices_ex_mask != (torch.tensor(indices_ex.shape) == 1).flatten()))
        if any((indices_ex_mask != (torch.tensor(indices_ex.shape) == 1).flatten())):
            indices_ex = indices_ex.expand(params.shape[:axis]
                                           + indices.shape[batch_dims:]
                                           + params.shape[axis + 1:])
        print("EXPANDED SHAPES: ", params_ex.shape, indices_ex.shape)
        return params_ex, indices_ex
    if batch_dims == 0:
        dim_diff = params.dim() - indices.dim()
        print(f"DIM_DIFF= {dim_diff}")
        if dim_diff != 0:
            params_expanded, indices_expanded = expand_p_i(params, indices)
            result = torch.gather(
                                    params_expanded, axis,
                                    indices_expanded.long(),
                                    sparse_grad=False, out=out
                                 )
            result = result.to(dtype = params.dtype)
            return result
        else:
            params_expanded, indices_expanded = expand_p_i(params, indices)
            print(f"PARAMS_EXPANDED, INDICES_EXPANDED (DIM_DIFF == 0) = {params_expanded}, {indices_expanded}")
            result = torch.gather(
                params_expanded, axis, indices_expanded.long(), sparse_grad=False, out=out)
            result = result.to(dtype = params.dtype)

    else:
        params_slices = torch.unbind(params, axis=0) if params.dim() > 1 else params
        indices_slices = torch.unbind(indices, axis=0) if indices.dim() > 1 else indices
        print(f"SLICES: {params_slices}, {indices_slices}")
        for b in range(batch_dims):
            if b == 0:
                zip_list = [(p, i) for
                            p, i in
                            zip(params_slices, indices_slices)
                            ]
            else:
                zip_list = [
                           (p, i) for z in [zip(p1, i1) for p1, i1 in zip_list]
                           for p, i in z
                            ]
        for z in zip_list:
            p, i = z
            dim_diff = p.dim() - i.dim()
            p_expanded = p
            i_expanded = i
            if dim_diff != 0:
              p_expanded, i_expanded = expand_p_i(p, i)
            print("p, i shapes: ", p_expanded.shape, i_expanded.shape)
            r = torch.gather(
                             p_expanded, (axis - batch_dims),
                             i_expanded.long(),
                             sparse_grad=False, out=None
                            )

            result.append(r)
        result = torch.stack(result)
        result = result.reshape(params.shape[:axis] + max(indices.shape[batch_dims:], torch.Size([1])) + params.shape[axis + 1:])
        result = result.to(dtype = params.dtype)
        print("FINAL RES SHAPE: ", result.shape, params.shape[:axis], indices.shape[batch_dims:], params.shape[axis + 1:])


    return result

In [314]:
params = torch.tensor([[-1.],
                 [ 1.],
                 [ 1.]])

indices = torch.tensor([[[0],
                  [0]],

                 [[0],
                  [0]],

                 [[0],
                  [0]]])

paramstf = tf.constant([[-1.],
                 [ 1.],
                 [ 1.]])

indicestf = tf.constant([[[0],
                  [0]],

                 [[0],
                  [0]],

                 [[0],
                  [0]]])

axis = 1
batch_dims = 1

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
torch_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor)
tf_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print()
print(torch_tensor)
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[[-1.]
  [-1.]]

 [[ 1.]
  [ 1.]]

 [[ 1.]
  [ 1.]]], shape=(3, 2, 1), dtype=float32)


AXIS: 1
SLICES: (tensor([-1.]), tensor([1.]), tensor([1.])), (tensor([[0],
        [0]]), tensor([[0],
        [0]]), tensor([[0],
        [0]]))
INDICES SHAPE:  torch.Size([2, 1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 1])
torch.Size([1]) torch.Size([1]) torch.Size([]) 1
SIZE TUPLES:  torch.Size([1]) torch.Size([]) torch.Size([1]) tensor([-1.])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1, 1]) Params shape:  torch.Size([1])
FINAL SHAPE:  torch.Size([1, 1]) torch.Size([1, 1]) torch.Size([2, 1])
tensor([1, 1]) tensor([True, True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([1, 1]) torch.Size([2, 1])
p, i shapes:  torch.Size([1, 1]) torch.Size([2, 1])
INDICES SH

In [311]:
params = torch.tensor([0])

indices = torch.tensor([0, 0])

paramstf = tf.constant([0])

indicestf = tf.constant([0, 0])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([0 0], shape=(2,), dtype=int32)


AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([2])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([2]) 
 torch.Size([1])
torch.Size([]) torch.Size([2]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([2]) torch.Size([1]) torch.Size([1]) tensor([0])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1]) Params shape:  torch.Size([1])
FINAL SHAPE:  torch.Size([2]) torch.Size([1]) torch.Size([2])
tensor([2]) tensor([True])
CONDITIONS FOR MASKS- PARAMS:  tensor([True])
CONDITIONS FOR MASKS- INDICES:  tensor([False])
EXPANDED SHAPES:  torch.Size([2]) torch.Size([2])
PARAMS_EXPANDED, INDICES_EXPANDED (DIM_DIFF == 0) = tensor([0, 0]), tensor([0, 0])
torch:  tensor([0, 0])
AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([2])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([2]) 
 torch.Size([1])
torch.Size([

In [282]:
params = torch.tensor([-1])

indices = torch.tensor([[0]])

paramstf = tf.constant([-1])

indicestf = tf.constant([[0]])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)


AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([1, 1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 1])
torch.Size([]) torch.Size([1, 1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1, 1]) torch.Size([1]) torch.Size([1]) tensor([-1])
True

Params insert shape: torch.Size([1, 1])  params reshape shape:  torch.Size([1, 1]) Params shape:  torch.Size([1])
FINAL SHAPE:  torch.Size([1, 1]) torch.Size([1, 1]) torch.Size([1, 1])
tensor([1, 1]) tensor([True, True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([1, 1]) torch.Size([1, 1])
torch:  tensor([[-1]])
AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([1, 1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 1])
torch.Size([])

In [283]:
params = torch.tensor([[-1]])

indices = torch.tensor([0])

paramstf = tf.constant([[-1]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)


AXIS: 0
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 1])
torch.Size([]) torch.Size([1]) torch.Size([1]) 0
SIZE TUPLES:  torch.Size([1]) torch.Size([1]) torch.Size([1, 1]) tensor([[-1]])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1, 1]) Params shape:  torch.Size([1, 1])
FINAL SHAPE:  torch.Size([1, 1]) torch.Size([1, 1]) torch.Size([1, 1])
tensor([1, 1]) tensor([True, True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([1, 1]) torch.Size([1, 1])
torch:  tensor([[-1]])
AXIS: 0
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 1])
torch.Size([]) t

In [284]:
params = torch.tensor([0])

indices = torch.tensor([0])

paramstf = tf.constant([0])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([0], shape=(1,), dtype=int32)


AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1]) 
 torch.Size([1])
torch.Size([]) torch.Size([1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1]) torch.Size([1]) torch.Size([1]) tensor([0])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1]) Params shape:  torch.Size([1])
FINAL SHAPE:  torch.Size([1]) torch.Size([1]) torch.Size([1])
tensor([1]) tensor([True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False])
CONDITIONS FOR MASKS- INDICES:  tensor([False])
EXPANDED SHAPES:  torch.Size([1]) torch.Size([1])
PARAMS_EXPANDED, INDICES_EXPANDED (DIM_DIFF == 0) = tensor([0]), tensor([0])
torch:  tensor([0])
AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1]) 
 torch.Size([1])
torch.Size([]) torch.S

In [285]:
params = torch.tensor([[0],
                 [0]])

indices = torch.tensor([0])

paramstf = tf.constant([[0],
                 [0]])

indicestf = tf.constant([0])

axis = 1
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[0]
 [0]], shape=(2, 1), dtype=int32)


AXIS: 1
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([2, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([2, 1])
torch.Size([1]) torch.Size([1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1]) torch.Size([1]) torch.Size([2, 1]) tensor([[0],
        [0]])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([2, 1]) Params shape:  torch.Size([2, 1])
FINAL SHAPE:  torch.Size([2, 1]) torch.Size([2, 1]) torch.Size([1, 1])
tensor([0, 1]) tensor([False,  True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([ True, False])
EXPANDED SHAPES:  torch.Size([2, 1]) torch.Size([2, 1])
torch:  tensor([[0],
        [0]])
AXIS: 1
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([2, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torc

In [312]:
params = torch.tensor([[-13712,  25013, -11631,  -2072,  -7720, -29788, -27294]])

indices = torch.tensor([3, 4, 1])

paramstf = tf.constant([[-13712,  25013, -11631,  -2072,  -7720, -29788, -27294]])

indicestf = tf.constant([3, 4, 1])

axis = 1
batch_dims = 0
print((torch.tensor(indices.shape[batch_dims:]) != params.shape[axis]))
print(f"BATCH DIMS: {batch_dims}")
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([True])
BATCH DIMS: 0
tf:  tf.Tensor([[-2072 -7720 25013]], shape=(1, 3), dtype=int32)


AXIS: 1
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([3])
PARAMS SHAPE:  torch.Size([1, 7])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 3]) 
 torch.Size([1, 7])
torch.Size([1]) torch.Size([3]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([3]) torch.Size([7]) torch.Size([1, 7]) tensor([[-13712,  25013, -11631,  -2072,  -7720, -29788, -27294]])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1, 1]) Params shape:  torch.Size([1, 7])
FINAL SHAPE:  torch.Size([1, 3]) torch.Size([1, 7]) torch.Size([1, 3])
tensor([1, 0]) tensor([ True, False])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([1, 7]) torch.Size([1, 3])
torch:  tensor([[-2072, -7720, 25013]])
AXIS: 1
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([3])
PARAMS SHAPE:  torch.Size([1, 7])
SINGLETON

In [287]:
params = torch.tensor([[-1, -1]])

indices = torch.tensor([0])

paramstf = tf.constant([[-1, -1]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0



print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[-1 -1]], shape=(1, 2), dtype=int32)


AXIS: 0
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1, 2])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 2])
torch.Size([]) torch.Size([1]) torch.Size([1]) 0
SIZE TUPLES:  torch.Size([1]) torch.Size([1]) torch.Size([1, 2]) tensor([[-1, -1]])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1, 2]) Params shape:  torch.Size([1, 2])
FINAL SHAPE:  torch.Size([1, 2]) torch.Size([1, 2]) torch.Size([1, 1])
tensor([1, 0]) tensor([ True, False])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False,  True])
EXPANDED SHAPES:  torch.Size([1, 2]) torch.Size([1, 2])
torch:  tensor([[-1, -1]])
AXIS: 0
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1, 2])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 2])
tor

In [288]:
params = torch.tensor([[-1]])

indices = torch.tensor([[0]])

paramstf = tf.constant([[-1]])

indicestf = tf.constant([[0]])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[[-1]]], shape=(1, 1, 1), dtype=int32)


AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([1, 1])
PARAMS SHAPE:  torch.Size([1, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 1, 1]) 
 torch.Size([1, 1, 1])
torch.Size([]) torch.Size([1, 1]) torch.Size([1]) 0
SIZE TUPLES:  torch.Size([1, 1]) torch.Size([1]) torch.Size([1, 1]) tensor([[-1]])
True

Params insert shape: torch.Size([1, 1])  params reshape shape:  torch.Size([1, 1, 1]) Params shape:  torch.Size([1, 1])
FINAL SHAPE:  torch.Size([1, 1, 1]) torch.Size([1, 1, 1]) torch.Size([1, 1, 1])
tensor([1, 1, 1]) tensor([True, True, True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False, False])
EXPANDED SHAPES:  torch.Size([1, 1, 1]) torch.Size([1, 1, 1])
PARAMS_EXPANDED, INDICES_EXPANDED (DIM_DIFF == 0) = tensor([[[-1]]]), tensor([[[0]]])
torch:  tensor([[[-1]]])
AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([1, 1])


In [ ]:
a = torch.tensor([[1]])
print(a.reshape((1, 1, 1)))

In [289]:
params = torch.tensor([[-1], [-1], [-1]])

indices = torch.tensor([1])

paramstf = tf.constant([[-1], [-1], [-1]])

indicestf = tf.constant([1])

axis = 0
batch_dims = 0
print(indices.unsqueeze(-1).unsqueeze(-1))
print(params.squeeze(-1))
print(params.dim(), indices.dim())
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([[[1]]])
tensor([-1, -1, -1])
2 1
tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)


AXIS: 0
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([3, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([3, 1])
torch.Size([]) torch.Size([1]) torch.Size([1]) 0
SIZE TUPLES:  torch.Size([1]) torch.Size([3]) torch.Size([3, 1]) tensor([[-1],
        [-1],
        [-1]])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1, 1]) Params shape:  torch.Size([3, 1])
FINAL SHAPE:  torch.Size([1, 1]) torch.Size([3, 1]) torch.Size([1, 1])
tensor([0, 1]) tensor([False,  True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([3, 1]) torch.Size([1, 1])
torch:  tensor([[-1]])
AXIS: 0
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([3, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
R

In [ ]:
torch.gather(torch.tensor([[-1], [-1], [-1]]), 0, torch.tensor([[1]]))

In [290]:
params = torch.tensor([[0], [0], [1]])

indices = torch.tensor([0])

paramstf = tf.constant([[0], [0], [1]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 1

print(params.dim(), indices.dim())
print(paramstf.shape[:axis] + indicestf.shape[batch_dims:] + paramstf.shape[axis + 1:])
print(params.shape[:axis] + indices.shape[batch_dims:] + params.shape[axis + 1:])

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

2 1
(1,)
torch.Size([1])
tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)


AXIS: 0
SLICES: (tensor([0]), tensor([0]), tensor([1])), tensor([0])
INDICES SHAPE:  torch.Size([])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([])
RESHAPE SHAPE:  torch.Size([]) 
 torch.Size([1])
torch.Size([]) torch.Size([]) torch.Size([]) 1
SIZE TUPLES:  torch.Size([]) torch.Size([1]) torch.Size([1]) tensor([0])
True

Params insert shape: torch.Size([])  params reshape shape:  torch.Size([]) Params shape:  torch.Size([1])
FINAL SHAPE:  torch.Size([]) torch.Size([]) torch.Size([])
tensor([]) tensor([], dtype=torch.bool)
CONDITIONS FOR MASKS- PARAMS:  tensor([], dtype=torch.bool)
CONDITIONS FOR MASKS- INDICES:  tensor([], dtype=torch.bool)
EXPANDED SHAPES:  torch.Size([]) torch.Size([])
p, i shapes:  torch.Size([]) torch.Size([])
FINAL RES SHAPE:  torch.Size([1, 1]) torch.Size([]) torch.Size([]) torch.Size([1])
torch:  tensor([[0]])
AXIS: 0
SLICES: (tensor([0]), tensor([0]), tens

In [291]:
params = torch.tensor([[0], [1]])

indices = torch.tensor([[0]])

paramstf = tf.constant([[0], [1]])

indicestf = tf.constant([[0]])



axis = 1
batch_dims = 0
print(torch.gather(params, axis, indices))
print(params.shape)
print(indices.expand((indices.shape[0], params.shape[-1 - axis])))
print(torch.gather(params, axis, indices))
print(params.dim(), indices.dim())
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([[0]])
torch.Size([2, 1])
tensor([[0, 0]])
tensor([[0]])
2 2
tf:  tf.Tensor(
[[[0]]

 [[1]]], shape=(2, 1, 1), dtype=int32)


AXIS: 1
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([1, 1])
PARAMS SHAPE:  torch.Size([2, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 1, 1]) 
 torch.Size([2, 1, 1])
torch.Size([1]) torch.Size([1, 1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1, 1]) torch.Size([1]) torch.Size([2, 1]) tensor([[0],
        [1]])
True

Params insert shape: torch.Size([1, 1])  params reshape shape:  torch.Size([2, 1, 1]) Params shape:  torch.Size([2, 1])
FINAL SHAPE:  torch.Size([2, 1, 1]) torch.Size([2, 1, 1]) torch.Size([1, 1, 1])
tensor([0, 1, 1]) tensor([False,  True,  True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([ True, False, False])
EXPANDED SHAPES:  torch.Size([2, 1, 1]) torch.Size([2, 1, 1])
PARAMS_EXPANDED, INDICES_EXPANDED (DIM_DIFF == 0) = tensor([[[0]],

       

In [190]:
axis = 1
batch_dims = 0

params = torch.tensor([[[0], [2]]])

indices = torch.tensor([[[0], [0]], [[0], [0]]])
print("xxx: ", params.shape[:2] + indices.shape + params.shape[2 + 1:])
#params = params.expand((1, 2, 2))
#params = params.unsqueeze(-1).unsqueeze(-1)
params = params.expand(params.shape[:axis] + indices.shape + params.shape[axis + 1:])
print("params shape: ", params.shape)
indices = indices.unsqueeze(0).unsqueeze(2)
indices = indices.expand(params.shape)

paramstf = tf.constant([[[0], [2]]])

indicestf = tf.constant([[[0], [0]], [[0], [0]]])
print(paramstf.shape, indicestf.shape)

print(torch.gather(params, axis, indices))
print(tf.gather(paramstf, indicestf, axis, batch_dims = batch_dims))

xxx:  torch.Size([1, 2, 2, 2, 1])


RuntimeError: ignored

In [95]:
a = torch.tensor([[1, 2]])
a.shape
a0 = a.expand((3, 3, 2))
a = a0.reshape((3, 3, 1, 2))
a = a.expand((3, 3, 2, 2))
ai0 = torch.ones((3, 2))
ai = ai0.reshape((1, 3, 2, 1))
ai = ai.expand((3, 3, 2, 2))

b = [1 for dim in list(a.shape)]
c = torch.Size(b)
d = torch.ones(c)

restorch = torch.gather(a, 1, ai.long())
torch_to_tf_a = tf.convert_to_tensor(a0, dtype = "int32")
torch_to_tf_ai = tf.convert_to_tensor(ai0, dtype = "int32")
print(torch_to_tf_a.shape, torch_to_tf_ai.shape)
restf = tf.gather(torch_to_tf_a, torch_to_tf_ai, axis = 1, batch_dims = 0)
print(restf)
print()
print(restorch.shape)
restf == tf.convert_to_tensor(restorch, dtype = "int32")


(3, 3, 2) (3, 2)
tf.Tensor(
[[[[1 2]
   [1 2]]

  [[1 2]
   [1 2]]

  [[1 2]
   [1 2]]]


 [[[1 2]
   [1 2]]

  [[1 2]
   [1 2]]

  [[1 2]
   [1 2]]]


 [[[1 2]
   [1 2]]

  [[1 2]
   [1 2]]

  [[1 2]
   [1 2]]]], shape=(3, 3, 2, 2), dtype=int32)

torch.Size([3, 3, 2, 2])


<tf.Tensor: shape=(3, 3, 2, 2), dtype=bool, numpy=
array([[[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]]])>

In [293]:
params = torch.tensor([1])

indices = torch.tensor([[0], [0]])

paramstf = tf.constant([1])

indicestf = tf.constant([[0], [0]])

axis = -1
batch_dims = 0

print(params.shape)
print(indices.expand((indices.shape[0], params.shape[-1 - axis])))
print(params.shape[:axis] , indices.shape[batch_dims:] , params.shape[axis + 1:])
print(params.dim(), indices.dim())
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print(tf_tensor == tf.convert_to_tensor(torch_tensor, dtype = "int32"))
print("torch: ", torch_tensor)
print()

torch.Size([1])
tensor([[0],
        [0]])
torch.Size([]) torch.Size([2, 1]) torch.Size([1])
1 2
tf:  tf.Tensor(
[[1]
 [1]], shape=(2, 1), dtype=int32)


AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([2, 1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([2, 1]) 
 torch.Size([1, 1])
torch.Size([]) torch.Size([2, 1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([2, 1]) torch.Size([1]) torch.Size([1]) tensor([1])
True

Params insert shape: torch.Size([1, 1])  params reshape shape:  torch.Size([1, 1]) Params shape:  torch.Size([1])
FINAL SHAPE:  torch.Size([2, 1]) torch.Size([1, 1]) torch.Size([2, 1])
tensor([2, 1]) tensor([True, True])
CONDITIONS FOR MASKS- PARAMS:  tensor([ True, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([2, 1]) torch.Size([2, 1])
torch:  tensor([[1],
        [1]])
AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([2, 1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON

In [ ]:
a = torch.tensor([[[1]]])
b = torch.tensor([-1, -1, -1])
axis = 0
batch_dims = 0
print(a.shape, b.shape)
print(a.reshape((1, 1, 1, 1)))
print(a.shape[:axis], b.shape, a.shape[axis + 1:])

a = a.expand((a.shape[:axis] + b.shape + a.shape[axis + 1:]))
print(a)

In [298]:
params = torch.tensor([0,
                 0])

indices = torch.tensor([[0]])

paramstf = tf.constant([0,
                 0])

indicestf = tf.constant([[0]])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)


AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([1, 1])
PARAMS SHAPE:  torch.Size([2])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 2])
torch.Size([]) torch.Size([1, 1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1, 1]) torch.Size([2]) torch.Size([2]) tensor([0, 0])
True

Params insert shape: torch.Size([1, 1])  params reshape shape:  torch.Size([1, 1]) Params shape:  torch.Size([2])
FINAL SHAPE:  torch.Size([1, 1]) torch.Size([1, 2]) torch.Size([1, 1])
tensor([1, 0]) tensor([ True, False])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([1, 2]) torch.Size([1, 1])
torch:  tensor([[0]])
AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([1, 1])
PARAMS SHAPE:  torch.Size([2])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 2])
torch.Size([

In [ ]:
a = torch.tensor([0])
print(a.dim())
a = torch.tensor([[0]])
print(a.dim())

1
2


In [299]:
params = torch.tensor([[0,
                 0]])

indices = torch.tensor([[0]])
print(torch.tensor([(torch.tensor(indices.shape[batch_dims:]) == params.shape[axis:axis + 1]) * params.shape[axis]]).long()
        + torch.tensor([torch.tensor(indices.shape[batch_dims:]) != params.shape[axis:axis + 1]]).long())
paramstf = tf.constant([[0,
                 0]])

indicestf = tf.constant([[0]])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([1])
tf:  tf.Tensor([[[0 0]]], shape=(1, 1, 2), dtype=int32)


AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([1, 1])
PARAMS SHAPE:  torch.Size([1, 2])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 1, 1]) 
 torch.Size([1, 1, 2])
torch.Size([]) torch.Size([1, 1]) torch.Size([1]) 0
SIZE TUPLES:  torch.Size([1, 1]) torch.Size([1]) torch.Size([1, 2]) tensor([[0, 0]])
True

Params insert shape: torch.Size([1, 1])  params reshape shape:  torch.Size([1, 1, 2]) Params shape:  torch.Size([1, 2])
FINAL SHAPE:  torch.Size([1, 1, 2]) torch.Size([1, 1, 2]) torch.Size([1, 1, 1])
tensor([1, 1, 0]) tensor([ True,  True, False])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False,  True])
EXPANDED SHAPES:  torch.Size([1, 1, 2]) torch.Size([1, 1, 2])
PARAMS_EXPANDED, INDICES_EXPANDED (DIM_DIFF == 0) = tensor([[[0, 0]]]), tensor([[[0, 0]]])
torch:  tensor([[[0, 0]]])
AXIS: 0
DIM_DIFF= 0
INDICES SH

In [300]:
params = torch.tensor([0, 0, 0])

indices = torch.tensor([0, 0, 0, 0])

paramstf = tf.constant([0, 0, 0])

indicestf = tf.constant([0, 0, 0, 0])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([0 0 0 0], shape=(4,), dtype=int32)


AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([4])
PARAMS SHAPE:  torch.Size([3])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([4]) 
 torch.Size([3])
torch.Size([]) torch.Size([4]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([4]) torch.Size([3]) torch.Size([3]) tensor([0, 0, 0])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1]) Params shape:  torch.Size([3])
FINAL SHAPE:  torch.Size([4]) torch.Size([3]) torch.Size([4])
tensor([0]) tensor([False])
CONDITIONS FOR MASKS- PARAMS:  tensor([False])
CONDITIONS FOR MASKS- INDICES:  tensor([False])
EXPANDED SHAPES:  torch.Size([3]) torch.Size([4])
PARAMS_EXPANDED, INDICES_EXPANDED (DIM_DIFF == 0) = tensor([0, 0, 0]), tensor([0, 0, 0, 0])
torch:  tensor([0, 0, 0, 0])
AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([4])
PARAMS SHAPE:  torch.Size([3])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([4]) 
 t

In [301]:
params = torch.tensor([[0, 0]])

indices = torch.tensor([0])

paramstf = tf.constant([[0, 0]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)


AXIS: 0
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1, 2])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 2])
torch.Size([]) torch.Size([1]) torch.Size([1]) 0
SIZE TUPLES:  torch.Size([1]) torch.Size([1]) torch.Size([1, 2]) tensor([[0, 0]])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1, 2]) Params shape:  torch.Size([1, 2])
FINAL SHAPE:  torch.Size([1, 2]) torch.Size([1, 2]) torch.Size([1, 1])
tensor([1, 0]) tensor([ True, False])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False,  True])
EXPANDED SHAPES:  torch.Size([1, 2]) torch.Size([1, 2])
torch:  tensor([[0, 0]])
AXIS: 0
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1, 2])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 2])
torch.Siz

In [302]:
params = torch.tensor([0])

indices = torch.tensor([[0, 0]])

paramstf = tf.constant([0])

indicestf = tf.constant([[0, 0]])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)


AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([1, 2])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 2]) 
 torch.Size([1, 1])
torch.Size([]) torch.Size([1, 2]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1, 2]) torch.Size([1]) torch.Size([1]) tensor([0])
True

Params insert shape: torch.Size([1, 1])  params reshape shape:  torch.Size([1, 1]) Params shape:  torch.Size([1])
FINAL SHAPE:  torch.Size([1, 2]) torch.Size([1, 1]) torch.Size([1, 2])
tensor([1, 2]) tensor([True, True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False,  True])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([1, 2]) torch.Size([1, 2])
torch:  tensor([[0, 0]])
AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([1, 2])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 2]) 
 torch.Size([1, 1])
torch.Size([

In [132]:
a = torch.tensor([[0, 0, 0]])
a = a.expand(torch.Size([2, 3]))
print(a)
(torch.tensor(a.shape) == 2) * torch.tensor(a.shape) != torch.tensor([2, 0])
torch.tensor(a.shape) == 2
print(a.type())
a = a.to(dtype = torch.int8)
a.dtype

tensor([[0, 0, 0],
        [0, 0, 0]])
torch.LongTensor


torch.int8

In [333]:
params = torch.tensor([0, 1])

indices = torch.tensor([[0, 0]])

paramstf = tf.constant([0, 1])

indicestf = tf.constant([[0, 0]])

axis = 0
batch_dims = 0
#print(torch.tensor(indices.shape[batch_dims:])!= params.shape[axis])
#params_insert_shape = ((torch.tensor(indices.shape[batch_dims:]) == params.shape[axis:axis + 1][0]) * params.shape[axis : axis + 1][0]).long() + (torch.tensor(indices.shape[batch_dims:]) != params.shape[axis : axis + 1][0]).long()
#params_insert_shape = torch.Size([dim for dim in params_insert_shape])
#print(params_insert_shape)

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)
torch.gather(params.reshape((1, 2)), axis, indices)

tf:  tf.Tensor([[0 0]], shape=(1, 2), dtype=int32)


AXIS: 0
torch:  []
AXIS: 0
torch_to_tf:  tf.Tensor([], shape=(0,), dtype=int32)

tf.Tensor(False, shape=(), dtype=bool)


tensor([[0, 1]])

In [304]:
params = torch.tensor([0, 0])

indices = torch.tensor([[[0], [0]]])

paramstf = tf.constant([0, 0])

indicestf = tf.constant([[[0], [0]]])

axis = -1
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[[0]
  [0]]], shape=(1, 2, 1), dtype=int32)


AXIS: 0
DIM_DIFF= -2
INDICES SHAPE:  torch.Size([1, 2, 1])
PARAMS SHAPE:  torch.Size([2])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1, 1])
RESHAPE SHAPE:  torch.Size([1, 2, 1]) 
 torch.Size([1, 1, 2])
torch.Size([]) torch.Size([1, 2, 1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1, 2, 1]) torch.Size([2]) torch.Size([2]) tensor([0, 0])
True

Params insert shape: torch.Size([1, 2, 1])  params reshape shape:  torch.Size([1, 2, 1]) Params shape:  torch.Size([2])
FINAL SHAPE:  torch.Size([1, 2, 1]) torch.Size([1, 2, 1]) torch.Size([1, 2, 1])
tensor([1, 0, 1]) tensor([ True, False,  True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False, False])
EXPANDED SHAPES:  torch.Size([1, 2, 1]) torch.Size([1, 2, 1])
torch:  tensor([[[0],
         [0]]])
AXIS: 0
DIM_DIFF= -2
INDICES SHAPE:  torch.Size([1, 2, 1])
PARAMS SHAPE:  torch.Size([2])
SINGLETON_DIMS: torch.Size

In [305]:

params = torch.tensor([[1],[0],[1]])

indices = torch.tensor([0])

paramstf = tf.constant([[1],[0],[1]])

indicestf = tf.constant([0])

axis = -1
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[1]
 [0]
 [1]], shape=(3, 1), dtype=int32)


AXIS: 1
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([3, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([3, 1])
torch.Size([1]) torch.Size([1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1]) torch.Size([1]) torch.Size([3, 1]) tensor([[1],
        [0],
        [1]])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([3, 1]) Params shape:  torch.Size([3, 1])
FINAL SHAPE:  torch.Size([3, 1]) torch.Size([3, 1]) torch.Size([1, 1])
tensor([0, 1]) tensor([False,  True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([ True, False])
EXPANDED SHAPES:  torch.Size([3, 1]) torch.Size([3, 1])
torch:  tensor([[1],
        [0],
        [1]])
AXIS: 1
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([3, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SH

In [306]:

params = torch.tensor([0], dtype = torch.uint8)

indices = torch.tensor([[0]], dtype = torch.int8)

paramstf = tf.constant([0])

indicestf = tf.constant([[0]])

axis = -1
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print("torch_tensor.type()= ", torch_tensor.type())
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)


AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([1, 1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([1, 1])
torch.Size([]) torch.Size([1, 1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1, 1]) torch.Size([1]) torch.Size([1]) tensor([0], dtype=torch.uint8)
True

Params insert shape: torch.Size([1, 1])  params reshape shape:  torch.Size([1, 1]) Params shape:  torch.Size([1])
FINAL SHAPE:  torch.Size([1, 1]) torch.Size([1, 1]) torch.Size([1, 1])
tensor([1, 1]) tensor([True, True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([1, 1]) torch.Size([1, 1])
torch:  tensor([[0]], dtype=torch.uint8)
AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([1, 1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 1]) 

In [307]:
params = torch.tensor([0], dtype = torch.uint8)

indices = torch.tensor([0], dtype = torch.int32)

paramstf = tf.constant([0])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print("torch_tensor.type()= ", torch_tensor.type())
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([0], shape=(1,), dtype=int32)


AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1]) 
 torch.Size([1])
torch.Size([]) torch.Size([1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1]) torch.Size([1]) torch.Size([1]) tensor([0], dtype=torch.uint8)
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1]) Params shape:  torch.Size([1])
FINAL SHAPE:  torch.Size([1]) torch.Size([1]) torch.Size([1])
tensor([1]) tensor([True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False])
CONDITIONS FOR MASKS- INDICES:  tensor([False])
EXPANDED SHAPES:  torch.Size([1]) torch.Size([1])
PARAMS_EXPANDED, INDICES_EXPANDED (DIM_DIFF == 0) = tensor([0], dtype=torch.uint8), tensor([0], dtype=torch.int32)
torch:  tensor([0], dtype=torch.uint8)
AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([

In [308]:
params = torch.tensor([[0], [0]])

indices = torch.tensor([0])

paramstf = tf.constant([[0], [0]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)


AXIS: 0
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([2, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([2, 1])
torch.Size([]) torch.Size([1]) torch.Size([1]) 0
SIZE TUPLES:  torch.Size([1]) torch.Size([2]) torch.Size([2, 1]) tensor([[0],
        [0]])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1, 1]) Params shape:  torch.Size([2, 1])
FINAL SHAPE:  torch.Size([1, 1]) torch.Size([2, 1]) torch.Size([1, 1])
tensor([0, 1]) tensor([False,  True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([2, 1]) torch.Size([1, 1])
torch:  tensor([[0]])
AXIS: 0
DIM_DIFF= 1
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([2, 1])
SINGLETON_DIMS: torch.Size([1, 1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1, 1]) 
 torch.Size([2, 1])
torc

In [309]:
params = torch.tensor([1])

indices = torch.tensor([0])

paramstf = tf.constant([1])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([1], shape=(1,), dtype=int32)


AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1]) 
 torch.Size([1])
torch.Size([]) torch.Size([1]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1]) torch.Size([1]) torch.Size([1]) tensor([1])
True

Params insert shape: torch.Size([1])  params reshape shape:  torch.Size([1]) Params shape:  torch.Size([1])
FINAL SHAPE:  torch.Size([1]) torch.Size([1]) torch.Size([1])
tensor([1]) tensor([True])
CONDITIONS FOR MASKS- PARAMS:  tensor([False])
CONDITIONS FOR MASKS- INDICES:  tensor([False])
EXPANDED SHAPES:  torch.Size([1]) torch.Size([1])
PARAMS_EXPANDED, INDICES_EXPANDED (DIM_DIFF == 0) = tensor([1]), tensor([0])
torch:  tensor([1])
AXIS: 0
DIM_DIFF= 0
INDICES SHAPE:  torch.Size([1])
PARAMS SHAPE:  torch.Size([1])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1])
RESHAPE SHAPE:  torch.Size([1]) 
 torch.Size([1])
torch.Size([]) torch.S

In [160]:
torch.gather(torch.tensor([1]), 0, torch.tensor([0]))

tensor([1])

In [324]:
params = torch.tensor([-54,   0,   1,   7, -60,   1])

indices = torch.tensor([[1, 0, 2, 2, 1, 1]])

paramstf = tf.constant([-54,   0,   1,   7, -60,   1])

indicestf = tf.constant([[1, 0, 2, 2, 1, 1]])

axis = 0
batch_dims = 0
print(torch.gather(torch.tensor([[-54,   0,   1,   7, -60,   1]]), axis, torch.tensor([[1, 0, 2, 2, 1, 1]])))

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([[  0, -54,   1,   1,   0,   0]])
tf:  tf.Tensor([[  0 -54   1   1   0   0]], shape=(1, 6), dtype=int32)


AXIS: 0
DIM_DIFF= -1
INDICES SHAPE:  torch.Size([1, 6])
PARAMS SHAPE:  torch.Size([6])
SINGLETON_DIMS: torch.Size([1]), torch.Size([1, 1])
RESHAPE SHAPE:  torch.Size([1, 6]) 
 torch.Size([1, 6])
torch.Size([]) torch.Size([1, 6]) torch.Size([]) 0
SIZE TUPLES:  torch.Size([1, 6]) torch.Size([6]) torch.Size([6]) tensor([-54,   0,   1,   7, -60,   1])
True

Params insert shape: torch.Size([1, 6])  params reshape shape:  torch.Size([1, 6]) Params shape:  torch.Size([6])
FINAL SHAPE:  torch.Size([1, 6]) torch.Size([1, 6]) torch.Size([1, 6])
tensor([1, 0]) tensor([ True, False])
CONDITIONS FOR MASKS- PARAMS:  tensor([False, False])
CONDITIONS FOR MASKS- INDICES:  tensor([False, False])
EXPANDED SHAPES:  torch.Size([1, 6]) torch.Size([1, 6])


RuntimeError: ignored